In [1]:
# credentials 
subscription_id = 'a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b'
resource_group = 'ressourcesecu'
workspace_name = 'deuxiemeespace'
compute_engine = 'jucalcul1'
cluster_name = 'jucalcul1'
dataset_name = 'trainingdataset'
experiment_name = "25082021"

In [2]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel
from raiwidgets import FairnessDashboard
from raiwidgets import ExplanationDashboard
from azureml.pipeline.core import Pipeline
from azureml.core import Environment
from azureml.pipeline.core import Pipeline, StepSequence
import datetime

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.33.0 (c:\users\jcharley\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.32.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.33.0 (c:\users\jcharley\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.32.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.33.0 (c:\users\jcharley\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.32.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (a

In [13]:
# register a model

ws = Workspace.get(subscription_id = subscription_id,
                   resource_group = resource_group,
                   name = workspace_name)

# model to use
model_name = "rf"                        
from azureml.core.model import Model
model = Model(ws, model_name)
print("---------model -------------")
print(model)

# environment to use
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")
print("---------model -------------")
print(sklearn_env)

---------model -------------
Model(workspace=Workspace.create(name='deuxiemeespace', subscription_id='a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b', resource_group='ressourcesecu'), name=rf, id=rf:13, version=13, tags={}, properties={})
---------model -------------
Environment(Name: AzureML-Tutorial,
Version: 75)


In [14]:
##Deploy as web service

#To build the correct environment for ACI, provide the following:
#- A scoring script to show how to use the model
#- A configuration file to build the ACI
#-The model you trained before

In [ ]:
 #Create scoring script
 #The init() function, which typically loads the model into a global object. This function is run only once when the Docker 
 # container is started.

#The run(input_data) function uses the model to predict a value based on the input data. Inputs and outputs to the run typically 
# use JSON for serialization and de-serialization, but other formats are supported.

In [15]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './outputs/model/saved_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Writing score.py


In [ ]:
#Create configuration file

In [18]:
aci_tags={"Data": "MNIST", "Framework" : "sklearn", "Purpose" : "ACI Test"}

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               auth_enabled=False, #Can be set on True!
                                               ssl_enabled=False,
                                               enable_app_insights=True,
                                               tags=aci_tags, 
                                               description='Predict MNIST with sklearn using Azure ML')

In [ ]:
#Deploy in ACI
#The following code goes through these steps:

#Create environment object containing dependencies needed by the model using the environment file (myenv.yml)
#Create inference configuration necessary to deploy the model as a web service using:
#The scoring file (score.py)
#envrionment object created in previous step
#Deploy the model to the ACI container.
#Get the web service HTTP endpoint.

In [22]:
import uuid
import datetime
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model


ws = Workspace.get(subscription_id = subscription_id,
                   resource_group = resource_group,
                   name = workspace_name)

model = Model(ws, 'rf')
# Environment to use
myenv = Environment.get(workspace=ws, name="AzureML-Tutorial")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

# Name of the service (should be unique)
service_name = 'firstdeploiement-' + datetime.datetime.now().strftime("%m%d%H")

service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

here
here
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-01 19:03:15+02:00 Creating Container Registry if not exists.
2021-09-01 19:03:15+02:00 Registering the environment.
2021-09-01 19:03:17+02:00 Use the existing image.
2021-09-01 19:03:17+02:00 Generating deployment configuration.
2021-09-01 19:03:18+02:00 Submitting deployment to compute.
2021-09-01 19:03:24+02:00 Checking the status of deployment sklearn-mnist-090119..
2021-09-01 19:07:20+02:00 Checking the status of inference endpoint sklearn-mnist-090119.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: be3fd0f6-69e5-4492-8874-0d383742d098
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sklearn-mnist-090119. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_418f04dee9f266fe033713cdf3f1f849 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more in

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: be3fd0f6-69e5-4492-8874-0d383742d098
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sklearn-mnist-090119. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_418f04dee9f266fe033713cdf3f1f849 locally. Please refer to https:

In [23]:
print(service.get_logs()) 

None


In [ ]:
print("Status=", service.state)
print("Scoring URL:", service.scoring_uri)
print(service.get_logs())